# Skiing by evolution

The skiing game has a reward which is more difficult in terms of learning.

The goal of the game is to avoid trees and pass through the gates, but the reward is given only at the end. Reward equals to -3..-7 for living and -500 * (missed gates) at the end.

Because of that, the agent can evaluate its behavior only at the end of the game, and not immediately after passing (or not passing) gates.

The standart Q-learning technique would require significant amount of training in this case, because 99.9% of the network's weight updates would be meaningless, as they correspond to that random living penalty.

Therefore, we use another approach to train the network: an evolutionary algorithm. We use mutation and selection based on sum of rewards to train the network, instead of gradient updates. The solution basically follows this scheme:
1. Create random set of NN's, each consisting of 2 convolutional and 2 fully-connected layers
2. Evaluate their fitness (i.e. sum of rewards)
3. Choose the best ones
4. Crossover and mutate them (possibly adding new neurons)
5. Repeat stage 2 for the result.

Our solution is inspired by a method called NEAT, used to play Mario: https://www.youtube.com/watch?v=qv6UVOQ0F44

The example on the video uses handcrafted features, but we use 2 convolutional layers trained the same way instead. This way, this approach (theoretically) can play any game of this kind, without any game-specific features or rewards

However, training was not enough, so now it is able only of going straight down (at the beginning it presses keys quite randomly).

# Info

почему 5x5?

1. http://cs231n.github.io/convolutional-networks/
2. https://code.google.com/p/cuda-convnet/wiki/LayerParams#Local_response_normalization_layer_(same_map)
3. https://github.com/BVLC/caffe/blob/master/examples/net_surgery.ipynb
4. https://arxiv.org/abs/1602.07261
5. 

# Imports

In [1]:
from six.moves import cPickle
import cv2
import numpy as np
from scipy.signal import convolve2d
import theano
import gym
from gym import wrappers
import theano
import theano.tensor as T
import lasagne

Using gpu device 0: GeForce GTX 980 (CNMeM is disabled, cuDNN 5105)
/home/etoestja/venv/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
# Resizing to black-white 42x42
def _process_frame42(frame):
    frame = frame[34:34+160, :160]
    # Resize by half, then down to 42x42 (essentially mipmapping). If
    # we resize directly we lose pixels that, when mapped to 42x42,
    # aren't close enough to the pixel boundary.
    frame = cv2.resize(frame, (80, 80))
    frame = cv2.resize(frame, (42, 42))
    frame = frame.mean(2)
    frame = frame.astype(np.float32)
    frame *= (1.0 / 255.0)
    frame = np.reshape(frame, [42, 42, 1])
    return frame

In [3]:
# Neural Evolution
class NeuralNetwork:
    conv1_size = 5
    conv2_size = 5
    evolution_probability = 0.97
    scale_factor = 1
    def __init__(self):
        self.conv1_filtr = np.random.standard_normal((self.conv1_size, self.conv1_size))
        self.conv2_filtr = np.random.standard_normal((self.conv2_size, self.conv2_size))
        self.dense1_weights = np.random.standard_normal((49, 100))
        self.dense2_weights = np.random.standard_normal((100, 3))
    def Convolve(self, compressed_observation):
        input_var = T.dmatrix('inputs')

        pooling = theano.function([input_var],
                                  theano.tensor.signal.pool.pool_2d(input_var, (2, 2), ignore_border=True))
        return pooling(convolve2d(pooling(convolve2d(compressed_observation, self.conv1_filtr , mode='valid')),
                          self.conv2_filtr, mode='valid'))
    def ForwardPropogate(self, compressed_observation):
        result_convolution = self.Convolve(compressed_observation)
        result_convolution = result_convolution.reshape(1, -1)
        dense1_output = result_convolution.dot(self.dense1_weights)
        dense1_activations = 1 / (1 + np.exp(- dense1_output))
        dense2_output = dense1_activations.dot(self.dense2_weights)
        dense2_activations =  1 / (1 + np.exp(- dense2_output))
        return dense2_activations
    def Evolution(self):
        new_network = NeuralNetwork()
        new_network.conv1_filtr = self.conv1_filtr +\
            ((np.random.standard_normal((self.conv1_size, self.conv1_size)) - self.evolution_probability) > 0) \
            * np.random.standard_normal((self.conv1_size, self.conv1_size)) * self.scale_factor
        new_network.conv2_filtr = self.conv2_filtr +\
            ((np.random.standard_normal((self.conv2_size, self.conv2_size)) - self.evolution_probability) > 0) \
            * np.random.standard_normal((self.conv2_size, self.conv2_size)) * self.scale_factor
        new_network.dense1_weights = self.dense1_weights +\
            ((np.random.standard_normal(self.dense1_weights.shape) - self.evolution_probability) > 0) \
            * np.random.standard_normal(self.dense1_weights.shape) * self.scale_factor
        new_network.dense2_weights = self.dense2_weights +\
            ((np.random.standard_normal(self.dense2_weights.shape) - self.evolution_probability) > 0) \
            * np.random.standard_normal(self.dense2_weights.shape) * self.scale_factor
        return new_network

In [4]:
# a = argmax_a Q(s,a)
def predict_action(observation, network):
    #return env.action_space.sample()
    compressed_observation = _process_frame42(observation)
    return np.argmax(network.ForwardPropogate(compressed_observation[:,:,0]))

In [8]:
# Play one game taking actions provided by the network
def PlayGame(env, network):
    observation = env.reset()
    done = False
    iteration, all_reward = 0, 0
    while not done:
        #env.render()
        action = predict_action(observation, network)
        observation, reward, done, info = env.step(action)
        all_reward += reward

        #if iteration % 500 == 0:
        print(str(iteration) + "... ")
        
        if all_reward < -30000 or iteration >= 8000:
            break
        
        iteration += 1

    print("Reward: ", all_reward)
    return all_reward

# Training

In [9]:
env = gym.make("Skiing-v0")
#env = wrappers.Monitor(env, "/tmp/gym-results", force = True)

[2017-02-06 11:52:36,971] Making new env: Skiing-v0


In [ ]:
network = NeuralNetwork()
print("Initializing...")
reward = PlayGame(env, network)
print("Init done")

Initializing...
0... 
1... 
2... 
3... 
4... 
5... 
6... 
7... 
8... 
9... 
10... 
11... 
12... 
13... 
14... 
15... 
16... 
17... 
18... 
19... 
20... 
21... 
22... 
23... 
24... 
25... 
26... 
27... 
28... 
29... 
30... 
31... 
32... 
33... 
34... 
35... 
36... 
37... 
38... 
39... 
40... 
41... 
42... 
43... 
44... 
45... 
46... 
47... 
48... 
49... 
50... 
51... 
Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/IPython/core/ultratb.py", line 1118, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python2.7/dist-packages/IPython/core/ultratb.py", line 300, in wrapped
    return f(*args, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/IPython/core/ultratb.py", line 345, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/usr/lib/python2.7/inspect.py", line 1044, in getinnerframes
    framelist.append((tb.tb_frame,) + 

ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.


Unfortunately, your original traceback can not be constructed.



IndexError: string index out of range

[2017-02-06 11:53:07,085] Uncaught exception, closing connection.
Traceback (most recent call last):
  File "/usr/lib/python2.7/dist-packages/zmq/eventloop/zmqstream.py", line 407, in _run_callback
    callback(*args, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/tornado/stack_context.py", line 275, in null_wrapper
    return fn(*args, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelbase.py", line 276, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelbase.py", line 228, in dispatch_shell
    handler(stream, idents, msg)
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelbase.py", line 413, in execute_request
    self._abort_queues()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelbase.py", line 626, in _abort_queues
    self._abort_queue(stream)
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelbase.py", line 649, in _abort_queue
    poller

In [ ]:
num_evolution_try = 3
iteration = 0
while reward < -6000:
    evolution_rewards = []
    evolution_networks = []
    
    for i in range(0, num_evolution_try):
        new_network = network.Evolution()
        evolution_networks += [new_network]
        print("Epoch {0} try {1}".format(iteration, i))
        evolution_rewards += [PlayGame(env, new_network)]

    i_max = np.argmax(evolution_rewards)
    if evolution_rewards[i_max] < reward:
        continue
    else:
        network = evolution_networks[i_max]
        
    iteration += 1

Epoch 0 try 0
0... 
500... 
('Reward: ', -9013.0)
Epoch 0 try 1
0... 
500... 
1000... 
1500... 
2000... 
2500... 
3000... 
3500... 
4000... 
4500... 
5000... 
5500... 
6000... 
('Reward: ', -30000.0)
Epoch 0 try 2
0... 
500... 
('Reward: ', -9013.0)
Epoch 1 try 0
0... 
500... 
1000... 
1500... 
2000... 
2500... 
3000... 
3500... 
4000... 
4500... 
5000... 
5500... 
6000... 
('Reward: ', -30000.0)
Epoch 1 try 1
0... 
500... 
1000... 
1500... 
2000... 
2500... 
3000... 
3500... 
4000... 
4500... 
5000... 
5500... 
6000... 
('Reward: ', -30000.0)
Epoch 1 try 2
0... 
500... 
('Reward: ', -9013.0)
Epoch 2 try 0
0... 
500... 
1000... 
1500... 
2000... 
2500... 
3000... 
3500... 


In [ ]:
PlayGame(env, network)

In [20]:
# Save and restore

def save_to_cPickle(file_name, obj):
    f = open(file_name + '.save', 'wb')
    cPickle.dump(obj, f, protocol=cPickle.HIGHEST_PROTOCOL)
    f.close()

def load_from_cPickle(file_name):
    f = open(file_name + '.save', 'rb')
    loaded_obj = cPickle.load(f)
    f.close()
    return loaded_obj

In [ ]:
save_to_cPickle("best_network", network)
new_net = load_from_cPickle("best_network1")